In [68]:
import cv2
import mediapipe as mp
import numpy as np
import os
from scipy.spatial.distance import cosine

In [70]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

In [72]:
def calculate_angle_3d(a, b, c):
    a, b, c = np.array(a), np.array(b), np.array(c)
    ba, bc = a - b, c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    return np.degrees(angle)

In [74]:
def extract_angles(landmarks):
    key_angles = {
        "left_shoulder": calculate_angle_3d(landmarks[11], landmarks[13], landmarks[15]),
        "right_shoulder": calculate_angle_3d(landmarks[12], landmarks[14], landmarks[16]),
        "left_elbow": calculate_angle_3d(landmarks[13], landmarks[15], landmarks[16]),
        "right_elbow": calculate_angle_3d(landmarks[14], landmarks[16], landmarks[15]),
        "left_hip": calculate_angle_3d(landmarks[23], landmarks[25], landmarks[27]),
        "right_hip": calculate_angle_3d(landmarks[24], landmarks[26], landmarks[28]),
        "left_knee": calculate_angle_3d(landmarks[25], landmarks[27], landmarks[28]),
        "right_knee": calculate_angle_3d(landmarks[26], landmarks[28], landmarks[27]),
    }
    return key_angles

In [76]:
def extract_pose_vector(landmarks):
    return np.array([coord for landmark in landmarks for coord in landmark])

In [78]:
def load_ideal_pose_vectors(dataset_path):
    ideal_vectors = {}
    for yoga_pose in os.listdir(dataset_path):
        pose_folder = os.path.join(dataset_path, yoga_pose)
        if not os.path.isdir(pose_folder):
            continue
        vectors_list = []
        for img_name in os.listdir(pose_folder):
            img_path = os.path.join(pose_folder, img_name)
            image = cv2.imread(img_path)
            if image is None:
                continue
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = pose.process(image_rgb)
            if results.pose_landmarks:
                landmarks = [(lm.x, lm.y, lm.z) for lm in results.pose_landmarks.landmark]
                vectors_list.append(extract_pose_vector(landmarks))
        if vectors_list:
            avg_vector = np.mean(vectors_list, axis=0)
            ideal_vectors[yoga_pose] = avg_vector
    return ideal_vectors

In [80]:
ideal_pose_vectors = load_ideal_pose_vectors("dataset")

C:\Users\SABAREESH\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [84]:
def compare_pose_similarity(user_vector, ideal_vector):
    similarity = 1 - cosine(user_vector, ideal_vector)
    return similarity * 100

In [86]:
def extract_landmarks(results):
    return [(lm.x, lm.y, lm.z) for lm in results.pose_landmarks.landmark]

In [98]:
def provide_text_feedback(joint, error_x, error_y):
    directions = []
    if abs(error_y) > 10:
        directions.append("Up" if error_y < 0 else "Down")
    if abs(error_x) > 10:
        directions.append("Left" if error_x > 0 else "Right")
    if directions:
        return f"Move {joint} {' and '.join(directions)}"
    return ""

In [100]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.flip(frame, 1)
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image)
    if results.pose_landmarks:
        landmarks = extract_landmarks(results)
        user_vector = extract_pose_vector(landmarks)
        user_input_pose = "vriksasana"
        if user_input_pose in ideal_pose_vectors:
            score = compare_pose_similarity(user_vector, ideal_pose_vectors[user_input_pose])
            cv2.putText(frame, f"Pose Score: {int(score)}%", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            if score >= 100:
                cv2.putText(frame, "Perfect Pose!", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            elif score >= 80:
                cv2.putText(frame, "Good Pose!", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            else:
                for idx, landmark in enumerate(results.pose_landmarks.landmark):
                    cx, cy = int(landmark.x * frame.shape[1]), int(landmark.y * frame.shape[0])
                    ideal_x, ideal_y = int(ideal_pose_vectors[user_input_pose][idx * 3] * frame.shape[1]), int(ideal_pose_vectors[user_input_pose][idx * 3 + 1] * frame.shape[0])
                    error_x, error_y = cx - ideal_x, cy - ideal_y
                    joint_name = list(extract_angles(landmarks).keys())[idx % 8]
                    color = (0, 255, 0) if abs(error_x) < 10 and abs(error_y) < 10 else (0, 0, 255)
                    mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, 
                                              landmark_drawing_spec=mp_drawing.DrawingSpec(color=color, thickness=2, circle_radius=4))
                    feedback_text = provide_text_feedback(joint_name, error_x, error_y)
                    if feedback_text:
                        cv2.putText(frame, feedback_text, (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                        break
    cv2.imshow('Yoga Pose Correction', frame)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()